# 문제 6

(kaggle 형 문제) prob4 를 바탕으로 아래 데이터셋을 만든다.

. prob6_train: DateHour 변수 기준으로 2021년 8월 14일 전(8월 14일 미포함) 데이터 (행의 수: 5256개)

. prob6_test: DateHour 변수 기준으로 2021년 8월 14일 이후(8월 14일 포함) 데이터 (행의 수: 744 개)

일 때, prob6_train으로 target을 예측하는 모델을 만들어, 

prob6_test의 target에 대한 MAE를 최소화하는 모델을 만든다. 

prob6_test의 예측 결과를 아래와 같은 형식으로 출력한다. 파일명은 answer6.csv 이다.

|DateHour|TotalHour|
|--------|---------|
|2021-08-14 00:00:00|102.607580|
|2021-08-14 01:00:00|94.078890|
....

※ 실제 상황에 가까운 연습을 위해, prob6_test의 target 값은 모른다는 상황을 가정하고 문제를 푼다.

## 데이터 전처리

In [11]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
#import mlxtend
import sys
#import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
statsmodels 0.11.1


In [12]:
df_elec = pd.read_csv('elec.csv', parse_dates=['Date', 'DateHour'])
df_info = pd.read_csv('info.csv')
df_info['Date'] = pd.to_datetime(df_info['Date'])

In [13]:
df_elec1 = df_elec.pivot(
    index='DateHour', 
    columns='Minute', 
    values='Value').reset_index() # index에 위치한 DateHour를 컬럼에 위치시킵니다
df_elec1

Minute,DateHour,15분,30분,45분,60분
0,2021-01-01 00:00:00,62,61,61,61
1,2021-01-01 01:00:00,96,93,116,113
2,2021-01-01 02:00:00,106,96,106,107
3,2021-01-01 03:00:00,92,110,110,109
4,2021-01-01 04:00:00,108,105,106,108
...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139
6164,2021-09-14 20:00:00,124,130,128,130
6165,2021-09-14 21:00:00,134,130,125,124
6166,2021-09-14 22:00:00,100,109,120,114


In [14]:
holi = pd.to_datetime(["2021-01-01", "2021-02-11", "2021-02-12", "2021-03-01", "2021-05-05", "2021-05-19", "2021-08-16"]).date

# pd.Series.dt accessor를 통해 파생 변수들을 만듭니다.
df_elec1 = df_elec1.assign(
    DayName = df_elec1['DateHour'].dt.weekday, 
    Hour = df_elec1['DateHour'].dt.hour,
    AM = (df_elec1['DateHour'].dt.hour >= 12).astype('int'),
    Weekend_yn = df_elec1['DateHour'].dt.weekday.isin([5, 6]).astype('int'),
    Holiday_yn = df_elec1['DateHour'].dt.date.isin(holi).astype('int'), # df_elec1['DateHour'].dt.date는 pd.Series 입니다.
    Avg = df_elec1[['15분', '30분', '45분', '60분']].mean(axis=1),
    TotalHour = df_elec1[['15분', '30분', '45분', '60분']].sum(axis=1),
)

In [15]:
df_info1 = df_info.fillna(0)

In [16]:
df_basetable1 = df_elec1.merge(
    df_info1, 
    left_on='DateHour', 
    right_on='Date', how='inner'
).drop(columns='Date') # DateHour가 컬럼에 있으므로 DateHour, Date 모두 존재하여, Date는 삭제했습니다.
display(df_basetable1.head())
df_basetable1.shape

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,Avg,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,61.25,245,0,-3.2,2.4,71,0.0,109.8,0.0,1.5
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,104.50,418,0,-4.5,1.5,77,0.0,109.8,0.0,1.5
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,103.75,415,0,-3.9,2.6,58,0.0,109.8,0.0,1.5
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,105.25,421,0,-4.1,2.6,56,0.0,109.8,0.0,1.5
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,106.75,427,0,-4.6,2.6,60,0.0,109.8,0.0,1.5


(6168, 20)

In [17]:
# shift를 이용합니다.
df_prob3 = df_basetable1.sort_values('DateHour').reset_index(drop=True)
df_prob3['target'] = df_prob3['TotalHour'].shift(-24)
df_prob4 = pd.concat(
    [df_prob3] + [
        df_prob3['TotalHour'].shift(24 * i).rename('lag_{}'.format(i)) for i in range(1, 7)
    ], axis=1
).dropna()
df_prob4.head()

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,전기요금(계절),공장인원,인건비,target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6
144,2021-01-07 00:00:00,22,22,22,25,3,0,0,0,0,...,109.8,0.000000,1.5,252.0,96.0,96.0,271.0,88.0,253.0,245.0
145,2021-01-07 01:00:00,26,22,22,22,3,1,0,0,0,...,109.8,0.369565,1.5,396.0,85.0,85.0,432.0,99.0,418.0,418.0
146,2021-01-07 02:00:00,22,21,21,22,3,2,0,0,0,...,109.8,0.000000,1.5,411.0,85.0,85.0,439.0,88.0,415.0,415.0
147,2021-01-07 03:00:00,22,22,22,23,3,3,0,0,0,...,109.8,0.629213,1.5,398.0,90.0,90.0,426.0,88.0,421.0,421.0
148,2021-01-07 04:00:00,22,26,26,23,3,4,0,0,0,...,109.8,2.865979,1.5,420.0,93.0,93.0,435.0,95.0,427.0,427.0


In [18]:
# shift를 사용하지 않고 target를 구하는 방법입니다. (값을 구하기만 하고, 따로 prob4로 만들지는 않았습니다.)
df_basetable1.join(
    df_basetable1[['DateHour','TotalHour']].assign(
        DateHour=lambda x: x['DateHour'] - pd.Timedelta(days=1)
    ).set_index('DateHour').rename(columns={'TotalHour': 'target'}),
    on='DateHour', how='inner'
)

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비,target
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,...,245,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5,253
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,...,418,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5,418
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,...,415,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5,415
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,...,421,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5,421
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,...,427,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5,427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,2021-09-13 19:00:00,162,160,148,122,0,19,1,0,0,...,592,2820,22.3,2.1,78,0.0,167.2,4.763514,1.5,613
6140,2021-09-13 20:00:00,113,122,122,126,0,20,1,0,0,...,483,14,22.2,1.3,78,0.0,167.2,0.028986,1.5,512
6141,2021-09-13 21:00:00,122,131,122,115,0,21,1,0,0,...,490,72,21.9,1.9,77,0.0,167.2,0.146939,1.5,513
6142,2021-09-13 22:00:00,97,108,122,113,0,22,1,0,0,...,440,11,21.7,1.4,77,0.0,167.2,0.025000,1.5,443


In [19]:
# 문제에서 제시한 데이터셋을 만듭니다.
df_prob6_train = df_prob4.loc[df_prob4['DateHour'] < pd.to_datetime('2021-08-14')]
df_prob6_test = df_prob4.loc[df_prob4['DateHour'] >= pd.to_datetime('2021-08-14')]
# 정답 target은 떼어냅니다.
s_kaggle_ans =  df_prob6_test.pop('target') # Kaggle의 정답으로 간주하겠습니다, 실제 시험에서는 공개 되지 않습니다.

# Kaggle형 풀이 단계

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

Step 4: 모델 개선 작업을 합니다.

# Step1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

In [20]:
# Step1-1: 검증 방법을 정합니다.
# 평가(Test, df_prob6_test)는 df_prob4에서 시점을 기준으로 하여 분리되었습니다. 
# 평가 데이터가 만들어지는 유형에 맞춰 검증셋을 구성합니다.
# 검증 방법으로 검증셋을 마찬가지로 시점을 기준으로 비슷한 분량 (1달치)로 구성합니다.
# → 시점 기준의 Hold-out 검증법
df_train = df_prob6_train.loc[df_prob6_train['DateHour'] < pd.to_datetime('2021-07-14')] 
df_valid = df_prob6_train.loc[df_prob6_train['DateHour'] >= pd.to_datetime('2021-07-14')]

# 평가셋에 있는 데이터를 모두 입력으로 사용합니다.
X_all = df_prob6_test.columns.tolist()

In [21]:
# 모델의 검증 결과를 남겨두기 위한 리스트입니다.
s_hist = []

In [22]:
# 공통 루틴 입니다.

from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Step1-2: 검증 루틴을 만듭니다.
# 모델 검증 루틴입니다. 
def eval_model(model_name, reg):
    reg.fit(df_train[X_all], df_train['target'])
    # 검증 결과를 보이고
    s = "{:.5f}, {:.5f}".format(
        mean_absolute_error(df_valid['target'], reg.predict(df_valid[X_all])), 
        mean_absolute_error(df_train['target'], reg.predict(df_train[X_all]))
    )
    print(s)
    # 기록합니다.
    s_hist.append(pd.Series([model_name, s], index=['model_name', 'result']))
    display(pd.DataFrame(s_hist).groupby('model_name').last())

# Step 3: 모델 선택 루틴을 만듭니다.
# 모델 선택 루틴입니다. 
def select_model(reg):
    reg.fit(df_prob6_train[X_all], df_prob6_train['target'])
    prd = reg.predict(df_prob6_test[X_all])
    pd.DataFrame({
        'DateHour': df_prob6_test['DateHour'],
        'TotalHour': prd
    }).to_csv('answer6.csv', index=None)
    return prd

# Step2: Baseline 만들기

In [23]:
# ColumnTransformer를 통해 다양한 처리과정을 통합시킵니다.
from sklearn.svm import SVR

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName']),
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]),
    ('pt', 'passthrough', ['AM'])
])
# make_pipeline을 통해 처리 단계와 모델을 연결 시키는 데이터 통로(pipeline)을 만듭니다. 
reg_svr = make_pipeline(ct, SVR(kernel='rbf', C=10, gamma=0.1))
eval_model('baseline', reg_svr)

146.31594, 59.58739


,result
model_name,
baseline,"146.31594, 59.58739"


# Step 3: 모델 선택 루틴을 만듭니다.

In [24]:
prd = select_model(reg_svr)
# 자가 채점
print("자가채점", mean_absolute_error(s_kaggle_ans, prd))

자가채점 28.966073501836416


# Step 4: 모델 개선 작업을 합니다.

In [32]:
# svr2 하이퍼파라미터를 찾아 봅니다.
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', categories='auto', sparse=False), ['DayName']),   # DayName을 가변수화 합니다.
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]), # ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)] 표준화
    ('pt', 'passthrough', ['AM']), # 'AM' 처리 없이 통과 시킵니다.
])
reg_svr2 = make_pipeline(ct, SVR(C=1000, kernel='rbf', gamma=0.03))
eval_model('svr2', reg_svr2)

126.06277, 47.94924


,result
model_name,
baseline,"146.31594, 59.58739"
svr2,"126.06277, 47.94924"


In [34]:
prd = select_model(reg_svr2)
# 자가 채점
print("자가채점", mean_absolute_error(s_kaggle_ans, prd))

자가채점 30.31321038374131


In [42]:
# SVR3: TotalHour와 lag1~6 까지의 변수에  PCA를 사용합니다.

from sklearn.svm import SVR
from sklearn.decomposition import PCA

# make_pipeline는 ColumnTransformer 안에서도 쓸 수 있습니다.
# StadardScaler → PCA (n_components=?) TO-DO 최적의 성분수를 구해봅니다.
ct = ColumnTransformer([
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=4)), 
         ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]), # TotalHour, lag_1 ~ 6 까지 표준화합니다.
    ('ohe', OneHotEncoder(categories='auto'), ['DayName']),  # DayName을 가변수화 합니다.
    ('pt', 'passthrough', ['AM'])  # 'AM' 처리 없이 통과 시킵니다.
])

reg_svr3 = make_pipeline(ct, SVR(kernel='rbf', gamma=0.3, C=1000))
eval_model('svr3', reg_svr3)

123.11705, 35.28951


,result
model_name,
baseline,"146.31594, 59.58739"
svr2,"132.06400, 53.30057"
svr3,"123.11705, 35.28951"


In [43]:
prd = select_model(reg_svr3)
# 자가 채점
print("자가채점", mean_absolute_error(s_kaggle_ans, prd))

자가채점 32.26425316112179


In [45]:
# 'baseline', 'svr2', 'svr3'로 Voting 앙상블 모델을 만듭니다.
from sklearn.ensemble import VotingRegressor

reg_vt = VotingRegressor([
    ('svr2', reg_svr2),
    ('svr3', reg_svr3)
])
eval_model('vt1', reg_vt)

126.08369, 43.13477


,result
model_name,
baseline,"146.31594, 59.58739"
svr2,"132.06400, 53.30057"
svr3,"123.11705, 35.28951"
vt1,"126.08369, 43.13477"


In [83]:
# GB: Gradient Boosting Tree

from sklearn.ensemble import GradientBoostingRegressor

X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in X_gb.split(',')]

# 별도의 전처리 없이 지정한 변수들을 사용합니다.
ct = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])

# 튜닝을 해봅니다.
# learning_rate=?, n_estimators = ?, max_depth = ?, min_samples_leaf = ?, random_state = 123
reg_gb = make_pipeline(
    ct,
    GradientBoostingRegressor(
        n_estimators = 500,
        learning_rate = 0.03,
        max_depth = 6,
        min_samples_leaf = 4, 
        loss = 'huber', # 손실함수: huber
        random_state = 123
    )
)
eval_model('gb', reg_gb)

108.00215, 36.26707


,result
model_name,
baseline,"146.31594, 59.58739"
gb,"108.00215, 36.26707"
lr,"164.29570, 85.11019"
rf,"108.22553, 60.96430"
svr2,"132.06400, 53.30057"
svr3,"123.11705, 35.28951"
vt1,"126.08369, 43.13477"
vt2,"107.93581, 44.22659"
xgb,"106.29032, 45.38133"


In [60]:
prd = select_model(reg_gb)
# 자가 채점
print("자가채점", mean_absolute_error(s_kaggle_ans, prd))

자가채점 53.21031115717798


In [67]:
# RF: Random Forest  

from sklearn.ensemble import RandomForestRegressor

X_rf = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_rf = [i.strip() for i in X_rf.split(',')]

ct = ColumnTransformer([
    ('pt', 'passthrough', X_rf)
])

# TO-DO: n_estimators=?, max_depth=? 튜닝을 해봅니다. 
reg_rf = make_pipeline(
    ct,
    RandomForestRegressor(n_estimators=100, max_depth=9, random_state=123)
)
eval_model('rf', reg_rf)

107.47241, 50.56328


,result
model_name,
baseline,"146.31594, 59.58739"
gb,"116.01903, 63.65279"
rf,"107.47241, 50.56328"
svr2,"132.06400, 53.30057"
svr3,"123.11705, 35.28951"
vt1,"126.08369, 43.13477"


In [69]:
prd = select_model(reg_rf)
# 자가 채점
print("자가채점", mean_absolute_error(s_kaggle_ans, prd))

자가채점 34.16130834952613


In [74]:
# XGB: XGBoost 

import xgboost as xgb

X_xgb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_xgb = [i.strip() for i in X_xgb.split(',')]

ct = ColumnTransformer([
    ('pt', 'passthrough', X_xgb)
])
# n_estimators=?, learning_rate=?, max_depth=? 을 튜닝해봅니다.
reg_xgb = make_pipeline(
    ct,
    xgb.XGBRegressor(n_estimators=500, learning_rate=0.02, max_depth=6, random_state=123)
)
eval_model('xgb', reg_xgb)

106.29032, 45.38133


,result
model_name,
baseline,"146.31594, 59.58739"
gb,"116.01903, 63.65279"
lr,"164.29570, 85.11019"
rf,"108.22553, 60.96430"
svr2,"132.06400, 53.30057"
svr3,"123.11705, 35.28951"
vt1,"126.08369, 43.13477"
vt2,"112.10262, 60.82934"
xgb,"106.29032, 45.38133"


In [80]:
prd = select_model(reg_xgb)
# 자가 채점
print("자가채점", mean_absolute_error(s_kaggle_ans, prd))

자가채점 44.10046746141167


In [71]:
from sklearn.linear_model import LinearRegression

# LinearRegression
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName']), # 'DayName' OneHotEncoder drop='first'
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]), # 'TotalHour', lag_1 ~ 6  StandardScaler
    ('pt', 'passthrough', ['AM']) # 'AM' 'passthrough'
])

reg_lr = make_pipeline(ct, LinearRegression())
eval_model('lr', reg_lr)

164.29570, 85.11019


,result
model_name,
baseline,"146.31594, 59.58739"
gb,"116.01903, 63.65279"
lr,"164.29570, 85.11019"
rf,"108.22553, 60.96430"
svr2,"132.06400, 53.30057"
svr3,"123.11705, 35.28951"
vt1,"126.08369, 43.13477"
xgb,"123.37020, 99.76594"


In [84]:
# VotingRegressor

from sklearn.ensemble import VotingRegressor

# Voting할 모델을 선별합니다.
reg_vt2 = VotingRegressor([
    #('baseline', reg_svr),
    #('svr2', reg_svr2),
    ('svr3', reg_svr3),
    ('gb', reg_gb),
    ('rf', reg_rf),
    ('xgb', reg_xgb),
    # ('lr', reg_lr)
])
# Voting 모델의 성능을 측정 
eval_model('vt2', reg_vt2)

107.04565, 41.50873


,result
model_name,
baseline,"146.31594, 59.58739"
gb,"108.00215, 36.26707"
lr,"164.29570, 85.11019"
rf,"108.22553, 60.96430"
svr2,"132.06400, 53.30057"
svr3,"123.11705, 35.28951"
vt1,"126.08369, 43.13477"
vt2,"107.04565, 41.50873"
xgb,"106.29032, 45.38133"
